In [1]:
import datetime
import random
import time

import numpy
import pandas
import scipy
import matplotlib.pyplot as plt
import IPython.display as disp

class Progress:
    def __init__(self, total):
        self.bar = disp.ProgressBar(total)
        self.progress = 0
        self.last = 0
    def inc(self):
        self.progress += 1
        if time.monotonic_ns() - self.last > 1e8:
            self.bar.progress = self.progress
            self.bar.update()
            self.last = time.monotonic_ns()
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_value, tb):
        self.bar.progress = self.progress
        self.bar.update()
        return False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!test -d dataset || unzip /content/drive/Shareddrives/'STAT4609 Netflix'/dataset.zip -d dataset

## Data preprocessing
Due to time and memory constraints,
we only use the first $10000$ movies for training and evaluation.
The data are loaded into a `numpy` array with the following structure:

| Movie ID | User ID | Rating | Year | Day of year | Weekday |
| :---: | :---: | :---: | :---: | :---: | :---: |

The movie and user IDs are reordered into $0, 1, \ldots$ for the ease of indexing.

In [4]:
class Counter:
    def __init__(self):
        self.cache = {}
    def resolve(self, value, max):
        if value in self.cache:
            return self.cache[value]
        id = len(self.cache)
        if id >= max:
            return None
        self.cache[value] = id
        return id
    def size(self):
        return len(self.cache)

user_counter = Counter()
movie_counter = Counter()

def read_ratings_file(data, path, row, progress):
    with open(path) as fh:
        for line in fh:
            line = line.rstrip("\r\n")
            if line.endswith(":"):
                movie_id = int(line[:-1])
                movie_id = movie_counter.resolve(movie_id, 10000)
                progress.inc()
                if movie_id is None:
                    break
            elif "," in line:
                user, rating, date = line.split(",")
                user = int(user)
                rating = int(rating)
                user_id = user_counter.resolve(user, 1000)
                if user_id is None:
                    continue
                y, m, d = date.split("-")
                date = datetime.date(int(y), int(m), int(d))
                data[row] = [movie_id, int(user_id), int(rating), y, date.timetuple().tm_yday, date.weekday()]
                row += 1
    return row

def read_ratings():
    # data = pandas.DataFrame(columns=["movie", "user", "rating"])#, "year", "day_of_year", "weekday"])
    # data = numpy.zeros([100498277, 6])
    data = numpy.zeros([1000000, 6])
    row = 0
    with Progress(10000) as progress:
        row = read_ratings_file(data, "dataset/combined_data_1.txt", row, progress)
        row = read_ratings_file(data, "dataset/combined_data_2.txt", row, progress)
        row = read_ratings_file(data, "dataset/combined_data_3.txt", row, progress)
        row = read_ratings_file(data, "dataset/combined_data_4.txt", row, progress)
    return data[:row]

data = read_ratings()

[============================================================] 10002/10000

Reformat the input data into a user-movie matrix, and randomly select $20\%$ of the available ratings as test data.
The missing data are imputed by movie mean rating.

In [5]:
movie_count = movie_counter.size()
user_count = user_counter.size()
ratings_mat = numpy.empty((user_count, movie_count))
ratings_mat[:] = numpy.nan

evaluation = []

def fill_ratings():
    with Progress(data.shape[0]) as progress:
        for i in range(data.shape[0]):
            user = int(data[i, 1])
            movie = int(data[i, 0])
            rating = data[i, 2]
    
            if random.randrange(0, 100) < 20:
                evaluation.append((user, movie, rating))
            else:
                ratings_mat[user, movie] = rating

            progress.inc()

    mean_ratings = numpy.nanmean(ratings_mat, axis=0)
    mean_ratings[numpy.isnan(mean_ratings)] = numpy.nanmean(mean_ratings)
    inds = numpy.where(numpy.isnan(ratings_mat))
    ratings_mat[inds] = numpy.take(mean_ratings, inds[1])
    return mean_ratings

mean_ratings = fill_ratings()

[============================================================] 508093/508093

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: Mean of empty slice


In [6]:
class KnnRecommend:
    def __init__(self, metric, aggregate, k):
        self.metric = metric
        self.aggregate = aggregate
        self.k = k
    def fit(self, ratings):
        raise Exception("Uninitialized function")
    def predict_rating(self, user_id, movie_id):
        raise Exception("Uninitialized function")
    def evaluate(self):
        raise Exception("Uninitialized function")

In [7]:
def _fit(self, ratings):
    # Compute the `distmat` matrix such that distmat[i][j] is the distance between users i and j
    print("Computing distance matrix")
    distmat = self.metric(ratings)
    
    print("Sorting distances")
    similarity = numpy.argsort(distmat, axis=1)

    print("Aggregating results")
    predicted_ratings = numpy.empty(ratings.shape)
    with Progress(user_count) as progress:
        for user in range(user_count):
            neighbours = similarity[user, 1:(self.k + 1)] # do not include self
            neighbour_ratings = ratings[neighbours, :]
            predicted_ratings[user, :] = self.aggregate(neighbour_ratings, distmat[user, neighbours])
            progress.inc()
    self.predicted_ratings = predicted_ratings

    return self
KnnRecommend.fit = _fit

In [8]:
def _predict_rating(self, user_id, movie_id):
    return self.predicted_ratings[user_id, movie_id]
KnnRecommend.predict_rating = _predict_rating

In [9]:
def _evaluate(self):
    print("Fitting model")
    self.fit(ratings_mat)
    mse = 0
    print("Evaluating test results")
    with Progress(len(evaluation)) as progress:
        for user, movie, rating in evaluation:
            mse += (self.predict_rating(user, movie) - rating) ** 2
            progress.inc()
    mse /= len(evaluation)
    rmse = numpy.sqrt(mse)
    print(f"k = {self.k}, metric = {self.metric.__name__}, agg = {self.aggregate.__name__}, rmse = {rmse}")
KnnRecommend.evaluate = _evaluate

In [10]:
class Metric:
    def discrete(ratings):
        """The discrete metric"""
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0] ** 2) as progress:
            for i in range(ratings.shape[0]):
                for j in range(ratings.shape[0]):
                    ret[i, j] = 0 if numpy.array_equal(ratings[i, :], ratings[j, :]) else 1
                    progress.inc()
        return ret

    def euclidean(ratings):
        """2-norm squared"""
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0]) as progress:
            for i in range(ratings.shape[0]):
                ret[i, :] = numpy.sum((ratings - ratings[i, :]) ** 2, axis=1)
                progress.inc()
        return ret / (16 * ratings.shape[1])

    def taxicab(ratings):
        """1-norm squared"""
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0]) as progress:
            for i in range(ratings.shape[0]):
                ret[i, :] = numpy.sum(numpy.abs(ratings - ratings[i, :]), axis=1)
                progress.inc()
        return ret / (4 * ratings.shape[1])
    
    def cosine_three(ratings):
        """Cosine similarity"""
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0] * (ratings.shape[0] + 1) / 2) as progress:
            corrected = ratings - 3
            for i in range(ratings.shape[0]):
                for j in range(i, ratings.shape[0]):
                    x = corrected[i, :]
                    y = corrected[j, :]
                    inner = numpy.inner(x, y)
                    norm_x = numpy.linalg.norm(x)
                    norm_y = numpy.linalg.norm(y)
                    corr = inner / norm_x / norm_y
                    ret[i, j] = corr
                    progress.inc()
        return (1 - ret) / 2
    
    def cosine_mean(ratings):
        """Cosine similarity"""
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0] * (ratings.shape[0] + 1) / 2) as progress:
            corrected = ratings - mean_ratings
            for i in range(ratings.shape[0]):
                for j in range(i, ratings.shape[0]):
                    x = corrected[i, :]
                    y = corrected[j, :]
                    inner = numpy.inner(x, y)
                    norm_x = numpy.linalg.norm(x)
                    norm_y = numpy.linalg.norm(y)
                    corr = inner / norm_x / norm_y
                    ret[i, j] = corr
                    progress.inc()
        return (1 - ret) / 2

    def pearson(ratings):
        """Pearson correlation rescaled to [0, 1]"""
        from scipy.stats.mstats import pearsonr
        ret = numpy.empty((ratings.shape[0], ratings.shape[0]))
        with Progress(ratings.shape[0] * (ratings.shape[0] + 1) / 2) as progress:
            for i in range(ratings.shape[0]):
                for j in range(i, ratings.shape[0]):
                    corr, _ = pearsonr(ratings[i, :], ratings[j, :])
                    ret[i, j] = corr
                    progress.inc()
        return (1 - ret) / 2

    def spearman(ratings):
        """Spearman correlation rescaled to [0, 1]"""
        from scipy.stats import spearmanr
        corrmat, _ = spearmanr(ratings, axis=1)
        return (1 - corrmat) / 2

In [11]:
class Aggregate:
    def baseline(ratings, dist):
        return mean_ratings

    def average(ratings, dist):
        return numpy.average(ratings, axis=0)

    def weighted(ratings, dist):
        corr = 1 - dist
        sum = numpy.sum(numpy.abs(corr))
        if numpy.all(corr < 1e-10):
            print("Warning: Numerical error detected, falling back to Aggregate.average")
            return Aggregate.average(ratings, dist)
        delta = (ratings - mean_ratings) * corr.reshape((corr.shape[0], 1))
        return mean_ratings + numpy.sum(delta, axis=0) / sum

In [12]:
KnnRecommend(Metric.discrete, Aggregate.baseline, 1).evaluate()

Fitting model
Computing distance matrix


[============================================================] 1000000/1000000

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 1, metric = discrete, agg = baseline, rmse = 1.1963762624238339


In [13]:
KnnRecommend(Metric.euclidean, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 1000/1000

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = euclidean, agg = average, rmse = 1.1719953069488978


In [14]:
KnnRecommend(Metric.taxicab, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 1000/1000

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = taxicab, agg = average, rmse = 1.1744850670205753


In [15]:
KnnRecommend(Metric.cosine_three, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = cosine_three, agg = average, rmse = 1.1790061859929075


In [16]:
KnnRecommend(Metric.cosine_mean, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = cosine_mean, agg = average, rmse = 1.1027836533184445


In [17]:
KnnRecommend(Metric.pearson, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = pearson, agg = average, rmse = 1.1983998188109357


In [18]:
KnnRecommend(Metric.spearman, Aggregate.average, 10).evaluate()

Fitting model
Computing distance matrix
Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = spearman, agg = average, rmse = 1.1968789096831038


In [19]:
KnnRecommend(Metric.euclidean, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 1000/1000

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = euclidean, agg = weighted, rmse = 1.1718550924500097


In [20]:
KnnRecommend(Metric.taxicab, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 1000/1000

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = taxicab, agg = weighted, rmse = 1.1741711570807052


In [21]:
KnnRecommend(Metric.cosine_three, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = cosine_three, agg = weighted, rmse = 1.178323341468338


In [22]:
KnnRecommend(Metric.cosine_mean, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = cosine_mean, agg = weighted, rmse = 1.1009416665900855


In [23]:
KnnRecommend(Metric.pearson, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix


[============================================================] 500500/500500.0

Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = pearson, agg = weighted, rmse = 1.1963136924278797


In [24]:
KnnRecommend(Metric.spearman, Aggregate.weighted, 10).evaluate()

Fitting model
Computing distance matrix
Sorting distances
Aggregating results


[============================================================] 1000/1000

Evaluating test results


[============================================================] 101506/101506

k = 10, metric = spearman, agg = weighted, rmse = 1.196881856675309
